# Pull Amygdala-mPFC gPPI Estimates

This script  pull gPPI betas & t-statistics averaged over 4 mPFC ROIs 

- Last edited March 2020
- Paul A. Bloom

In [1]:
import nilearn
from nilearn import image
from nilearn import plotting
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import glob
import re

# Define subject frame

In [2]:
# data frame of all runs in feat
subFrame = pd.read_csv('../QA/allEmotionsFeatComplete.csv')
subFrame = subFrame[subFrame['runType'] == 'fear']

## Determine here which roi masks to pull estimates for, then flatten them

In [16]:
maskPaths = glob.glob('../Structural/mni2mmSpace/*.nii.gz')

maskPaths = [maskPaths[9], maskPaths[3], maskPaths[4], maskPaths[8]]
masks = ['none'] * len(maskPaths)
for counter, maskPath in enumerate(maskPaths):
    masks[counter] = image.load_img(maskPath)
    masks[counter] = masks[counter].get_fdata()
    masks[counter] = masks[counter].flatten()

maskPaths

['../Structural/mni2mmSpace/mPFC_sphere_5mm_mni2mm.nii.gz',
 '../Structural/mni2mmSpace/mPFC_sphere_5mm_anterior_down_mni2mm.nii.gz',
 '../Structural/mni2mmSpace/vmpfc_trimmed_prob.5_mni2mm.nii.gz',
 '../Structural/mni2mmSpace/mPFC_sphere_5mm_anterior_mni2mm.nii.gz']

# Define a function to get PPI for a certain ROI

In [18]:
def getPpiRoi(subject, roiMask, runType, ppiDir, copeNum, imageType):

    # Pull for the emotional face and neutral face cope for each run
    statImg = image.load_img('/danl/SB/Investigators/PaulCompileTGNG/data/%s/model/%s/24motion.feat/%s/reg_standard/stats/%s%s.nii.gz'%(subject, runType, ppiDir, imageType, copeNum))
    statImg = statImg.get_fdata()
    statImg = statImg.flatten()
    
    # Mask the cope images and filter based on mask == 1, then take means      
    maskedStatImg= statImg[roiMask == 1]
    meanStatImg = np.mean(maskedStatImg)
    
    # return value
    return(meanStatImg)

# Pull gPPI estimates with deconvolution

For 4 contrasts,and get both t-stats and beta estimates (copes)
* PHYSIO regressor
* emotion > baseline ppi
* neutral > baseline ppi
* emotion > neutral ppi

In [20]:
ppiDir = 'gppi_deconvolution_seed_harvardoxfordsubcortical_bilateralamyg.feat'

for index, row in subFrame.iterrows():
    for counter, roiMask in enumerate(masks):
        maskPath = maskPaths[counter]
        maskPathSplit = maskPath.split('/')
        roiName = maskPathSplit[3][:-14]
        if(os.path.isfile('/danl/SB/Investigators/PaulCompileTGNG/data/%s/model/%s/24motion.feat/%s/reg_standard/stats/cope1.nii.gz'%(row['name'], row['runType'], ppiDir))):
            subFrame.loc[index, 'phys_%s_cope'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 8, 'cope')
            subFrame.loc[index, 'emotPpi_%s_cope'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 9, 'cope')
            subFrame.loc[index, 'neutralPpi_%s_cope'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 10, 'cope')
            subFrame.loc[index, 'emotMinusNeutralPpi_%s_cope'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 11, 'cope')
            subFrame.loc[index, 'phys_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 8, 'tstat')
            subFrame.loc[index, 'emotPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 9, 'tstat')
            subFrame.loc[index, 'neutralPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 10, 'tstat')
            subFrame.loc[index, 'emotMinusNeutralPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 11, 'tstat')

## Export to CSV

In [6]:
csvname = 'bilateral_amyg_' + ppiDir + '_ppi_roi.csv'
subFrame.to_csv(csvname)

# Pull gPPI estimates without deconvolution

In [22]:
ppiDir = 'gppi_no_deconvolution_seed_harvardoxfordsubcortical_bilateralamyg.feat'

for index, row in subFrame.iterrows():
    for counter, roiMask in enumerate(masks):
        maskPath = maskPaths[counter]
        maskPathSplit = maskPath.split('/')
        roiName = maskPathSplit[3][:-14]
        if(os.path.isfile('/danl/SB/Investigators/PaulCompileTGNG/data/%s/model/%s/24motion.feat/%s/reg_standard/stats/cope1.nii.gz'%(row['name'], row['runType'], ppiDir))):
            subFrame.loc[index, 'phys_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 8, 'tstat')
            subFrame.loc[index, 'emotPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 9, 'tstat')
            subFrame.loc[index, 'neutralPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 10, 'tstat')
            subFrame.loc[index, 'emotMinusNeutralPpi_%s_tstat'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 11, 'tstat')
            subFrame.loc[index, 'phys_%s_cope'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 8, 'cope')
            subFrame.loc[index, 'emotPpi_%s_cope'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 9, 'cope')
            subFrame.loc[index, 'neutralPpi_%s_cope'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 10, 'cope')
            subFrame.loc[index, 'emotMinusNeutralPpi_%s_cope'%(roiName)] = getPpiRoi(row['name'],roiMask,row['runType'], ppiDir, 11, 'cope')

## Export to CSV

In [7]:
csvname = 'bilateral_amyg_' + ppiDir + '_ppi_roi.csv'
print(csvname)
subFrame.to_csv(csvname)